# 베드락을 이용하여 자료 요약하기

> *이 노트북은 SageMaker Studio*의 `Data Science 3.0` 커널과 잘 작동해야 합니다.

## 소개

이 노트북에서는 리테일 고객이 크기가 큰 문서를 요약하는 방법을 보여드리겠습니다.대용량 문서로 작업할 때 입력 텍스트가 모델 컨텍스트 길이에 맞지 않거나, 모델이 대용량 문서를 인식하지 못하거나, 메모리 부족 오류 등으로 인해 몇 가지 문제에 직면할 수 있습니다. 이러한 문제를 해결하기 위해 청킹 및 연쇄 프롬프트 개념을 기반으로 하는 아키텍처를 보여드리겠습니다. 이 아키텍처는 언어 모델로 구동되는 애플리케이션을 개발하는 데 널리 사용되는 프레임워크인 LangChain을 활용합니다.

[LangChain](https://python.langchain.com/docs/get_started/introduction.html)은 언어 모델로 구동되는 애플리케이션을 개발하기 위한 프레임워크입니다. 이 프레임워크의 핵심 측면을 통해 다양한 구성 요소를 연결하여 고급 사용 사례를 만들어 대규모 언어 모델을 보강할 수 있습니다.


**참고:** *이 노트북은 AWS 환경 내부 또는 외부에서 실행할 수 있습니다*.

#### 컨텍스트

이 노트북에서는 LangChain 프레임워크 내에서 Amazon Bedrock과 통합하여 사용는 방법과 PromptTemplate의 도움으로 텍스트를 생성하는 데 어떻게 사용될 수 있는지 살펴보겠습니다.

#### 패턴

![bedrock_langchian_image](./image/bedrock_summ.png)

이 아키텍처에서는
1. 큰 문서(또는 작은 문서를 추가한 거대한 파일)가 로드됩니다.
2. Langchain 유틸리티를 사용하여 여러 개의 작은 청크로 분할(청킹)합니다.
3. 첫 번째 청크가 모델로 전송되고, 모델은 해당 요약을 반환합니다.
4. 랭체인은 다음 청크를 가져와서 반환된 요약에 추가하고 결합된 텍스트를 모델에 새 요청으로 보냅니다; 모든 청크가 처리될 때까지 이 과정이 반복됩니다.
5. 결국, 전체 콘텐츠를 기반으로 한 최종 요약이 생성됩니다.

#### 사용사례

이 접근 방식은 통화 녹취록, 회의 녹취록, 책, 기사, 블로그 게시물 및 상품/서비스 관련 콘텐츠를 요약하는 데 사용할 수 있습니다.

#### 페르소나

리테일 회사인 애니컴퍼니의 제품 담당 매니저인 "김소매"는 최근 등록된 제품들에 대한 고객들의 리뷰를 확인하고 있습니다. 수백 개의 리뷰를 요약해서 몇 줄로 정리하여 '대표리뷰'로 등록을 하려고 하는데요, 하루에도 한 제품 당 수십개 이상 올라오는 리뷰를 매번 다 읽어 보기는 어렵습니다. 또한 시간이 경과하면서 바뀌는 고객의 리뷰를 파악하기도 쉽지 않습니다. 이에,대용량의 문서를 요약하여 보여 주기 위헤 LLM의 도움이 필요합니다.


#### 구현 방법

이 사용 사례를 보여주기 위해 이 노트북에서는 고객의 이전 제품 설명을 기반으로 신규 제품 설명을 생성하는 방법을 보여드리며, Boto3 클라이언트와 함께 Amazon Bedrock API를 사용하는 Anthropic Claude 모델을 사용하겠습니다.


## Setup

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) 베드락에 연결해야 합니다.

우선 사전에 설치가 필요한 패키지들을 설치하세요. 그 이후에 셋업에 필요한 라이브러리들을 설치합니다. 

In [2]:
%pip install -r ./dependencies/requirements.txt > /dev/null 2>&1

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import boto3

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"  # E.g. "us-east-1"
os.environ["BEDROCK_ENDPOINT_URL"] = "https://bedrock-runtime.us-east-1.amazonaws.com"  # E.g. "https://..."

session = boto3.Session(
    profile_name=os.environ.get("AWS_PROFILE")
) # sets the profile name to use for AWS credentials

bedrock = session.client(
    service_name='bedrock-runtime', # creates a Bedrock client
    region_name=os.environ.get("AWS_DEFAULT_REGION"),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL")
) 

## 베드락 LLM 모델 호출하기

LLM에서 Bedrock 클래스의 인스턴스를 생성하는 것으로 시작하겠습니다. 여기에는 Amazon Bedrock에서 사용할 수 있는 모델의 ARN인 model_id가 필요합니다.

선택적으로 이전에 생성한 boto3 클라이언트를 전달할 수 있으며, `temperature`, `top_p`, `max_tokens_to_sample` 또는 `stop_sequences`와 같은 매개 변수를 보유할 수 있는 일부 `model_kwargs`도 전달할 수 있습니다(매개 변수에 대한 자세한 내용은 Amazon Bedrock 콘솔에서 살펴볼 수 있습니다).

Amazon Bedrock에서 사용 가능한 텍스트 생성 모델 ID에 대한 [설명서](https://docs.aws.amazon.com/ko_kr/bedrock/latest/userguide/model-ids-arns.html)를 확인하세요.

모델마다 지원하는 `model_kwargs`가 다르다는 점에 유의하세요.

In [4]:
from langchain.llms.bedrock import Bedrock

model_kwargs = {'max_tokens_to_sample':8000, 
                "temperature":0,
                "top_k":250,
                "top_p":1,
                "stop_sequences": ["\n\nHuman:"]
                }

llm = Bedrock(model_id = "anthropic.claude-instant-v1",
#llm = Bedrock(model_id = "anthropic.claude-v2",
                    client = bedrock, 
                    model_kwargs = model_kwargs 
                    )


PDF 문서를 로드한 후에, 긴 문서를 청킹(chunking)합니다. 

In [5]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain


pdf_path = "./2022-Shareholder-Letter-ko.pdf"

loader = PyPDFLoader(file_path=pdf_path)
documents = loader.load()
        
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], chunk_size=4000, chunk_overlap=100 
)
    
docs = text_splitter.split_documents(documents=documents)


## MAP-REDUCE 

'map_reduce' 체인은 큰 문서를 관리하기 쉬운 작은 덩어리로 분할하여 문서 처리를 처리하도록 설계되었습니다. 이 체인은 각 조각에 초기 프롬프트를 사용하여 문서의 특정 섹션을 기반으로 요약 또는 답변을 생성합니다. 또한 MapReduceDocumentsChain은 생성된 출력을 가져와 다른 프롬프트를 사용하여 결합하여 전체 문서에 대한 포괄적이고 일관된 요약 또는 답변을 생성합니다. load_summarize_chain 함수를 사용하여 'map_reduce' 체인을 설정하고 출력 요약을 얻습니다.

In [6]:
from langchain.prompts import PromptTemplate
import textwrap
                                       
map_prompt_template = "{text}\n\n위의 내용을 Korean으로 bullet point 3개로 요약합니다:"
map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

combine_prompt_template = "{text}\n\n위의 내용을 Korean으로 간결하게 bullet point 5개로 요약합니다:"
combine_prompt = PromptTemplate(template=combine_prompt_template, input_variables=["text"])

chain = load_summarize_chain(llm, chain_type="map_reduce", map_prompt=map_prompt, combine_prompt=combine_prompt, verbose=True)

output_summary = chain.run(docs)

wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
CEO로서 두 번째 연례 주주 서한을 작성하기 위해 자리에 앉았을 때 저는 Amazon의 앞날에 대해 낙관적이고 활력을 얻었습니다. 2022년은 최근 기억에 있어 어려운 거시경제적 해 중 하나이고 우리 자체의 운영상의 어려움에도 불구하고 우리는 여전히 수요를 늘릴 방법을 찾았습니다(팬데믹 전반기에 경험한 전례 없는 성장에 더해). 우리는 장단기적으로 고객 경험을 의미 있게 개선하기 위해 대규모 사업을 혁신했습니다. 그리고 고객, 주주 및 직원을 위해 Amazon의 미래를 바꿀 수 있다고 믿는 장기 투자를 계속 유지하면서 투자 결정과 앞으로 나아갈 방법에 중요한 조정을 했습니다.  작년에 이례적인 수의 동시 도전 과제가 있었지만 현실은 유능하고 자금이 충분한 경쟁자가 많은 크고 역동적인 글로벌 시장 부문에서 운영하는 경우(아마존이 모든 비즈니스를 운영하는 조건) 조건이 오랫동안 정체되어 있는 경우는 드뭅니다.  제가 Amazon에 근무한 25년 동안 끊임없는 변화가 있었고 그 중 대부분은 우리가 스스로 시작했습니다. 내가 1997년에 Amazon에 입사했을 때 우리는 1996년에 1,500만 달러의 매출을 올렸고, 도서 전용 소매업체였으며, 제3자 시장이 없었고, 미국 내 주소로만 배송되었습니다. 오늘날 Amazon은 단위 판매의 60%를 차지하고 전 세계 거의 모든 국가의 고객에게 도달하는 활기찬 타사 판매자 에코시스템을 통해 상상할 수 있는 거의 모든 물리적 및 디지털 소매 품목을 판매합니다. 마찬가지로 클라우드에서 일련의 기술 인프라 서비스를 중심으로 비즈니스를 구축하는 것은 AWS를 추구하기 시작한 2003년에는 분명하지 않았으며 2006년 첫 서비스를 출시했을 때도 마찬가지였습니다. 거의 모든 책을 60초 안에 손끝에서 볼 수 있습니다 2007년 Kindle을

## Streamlit 어플리케이션 수행하기

#### (1) 어플리케이션 수행을 위한 스크립트 준비하기

In [7]:
%%writefile summarize_lib.py

import os
from langchain.prompts import PromptTemplate
from langchain.llms.bedrock import Bedrock
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

def get_llm():
    
    model_kwargs =  { #Anthropic 모델
        "max_tokens_to_sample": 8000,
        "temperature": 0, 
        "top_k": 250, 
        "top_p": 0.5, 
        "stop_sequences": ["\n\nHuman:"] 
    }
    
    llm = Bedrock(
        credentials_profile_name=os.environ.get("AWS_PROFILE"), #AWS 자격 증명에 사용할 프로필 이름을 설정합니다(기본값이 아닌 경우)
        region_name=os.environ.get("AWS_DEFAULT_REGION"), #리전 이름을 설정합니다(기본값이 아닌 경우)
        endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL"), #엔드포인트 URL 설정(필요한 경우)
        #model_id="anthropic.claude-v2", #파운데이션 모델 설정하기
        model_id="anthropic.claude-instant-v1", #파운데이션 모델 설정하기
        model_kwargs=model_kwargs) #Claude의 속성을 구성합니다.
    
    return llm

pdf_path = "uploaded_file.pdf"

def get_example_file_bytes(): #사용자가 기존 생성된 예제를 다운로드할 수 있도록 파일 바이트를 제공합니다.
    with open("2022-Shareholder-Letter-ko.pdf", "rb") as file:
        file_bytes = file.read()
    
    return file_bytes


def save_file(file_bytes): #업로드한 파일을 디스크에 저장하여 나중에 요약합니다.   
    with open(pdf_path, "wb") as f: 
        f.write(file_bytes)
    
    return f"Saved {pdf_path}"


def get_docs():
    
    loader = PyPDFLoader(file_path=pdf_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n"], chunk_size=4000, chunk_overlap=100 
    )
    docs = text_splitter.split_documents(documents=documents)
    
    return docs

def get_summary(return_intermediate_steps=False):
    
    map_prompt_template = "{text}\n\n위의 내용을 Korean으로 bullet point 3개로 요약합니다:"
    map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])
    
    combine_prompt_template = "{text}\n\n위의 내용을 Korean으로 간결하게 bullet point 5개로 요약합니다:"
    combine_prompt = PromptTemplate(template=combine_prompt_template, input_variables=["text"])
    
    llm = get_llm()
    docs = get_docs()
    
    chain = load_summarize_chain(
        llm, chain_type="map_reduce", 
        map_prompt=map_prompt, 
        combine_prompt=combine_prompt, 
        return_intermediate_steps=return_intermediate_steps,
        verbose=True
    )
    
    if return_intermediate_steps:
        return chain({"input_documents": docs}, return_only_outputs=True) #반환 구조를 chain.run(docs)와 일관성 있게 만들기
    else:
        return chain.run(docs)


Overwriting summarize_lib.py


In [8]:
%%writefile summarize_app.py

import streamlit as st
import summarize_lib as glib

st.set_page_config(layout="wide", page_title="문서 요약")
st.title("문서 요약")

uploaded_file = st.file_uploader("Select a PDF", type=['pdf'])

upload_button = st.button("Upload", type="primary")

if upload_button:
    with st.spinner("Uploading..."):
        
        upload_response = glib.save_file(file_bytes=uploaded_file.getvalue())

        st.success(upload_response)
        
        st.session_state.has_document = True

if 'has_document' in st.session_state: #문서가 업로드되었는지 확인하기
    
    return_intermediate_steps = st.checkbox("중간 단계 요약 보기", value=True)
    summarize_button = st.button("요약하기", type="primary")
    
    if summarize_button:
        st.subheader("통합 요약")

        with st.spinner("Running..."):
            response_content = glib.get_summary(return_intermediate_steps=return_intermediate_steps)


        if return_intermediate_steps:

            st.write(response_content["output_text"])

            st.subheader("중간 단계 요약")

            for step in response_content["intermediate_steps"]:
                st.write(step)
                st.markdown("---")

        else:
            st.write(response_content)

Overwriting summarize_app.py


In [15]:
%%writefile summ-setup.sh

pip install --no-cache-dir -r ./dependencies/requirements.txt
pip install anthropic
sudo yum install -y jq

Overwriting summ-setup.sh


In [16]:
%%writefile summ-run.sh

#!/bin/sh
CURRENTDATE=`date +"%Y-%m-%d %T"`
RED='\033[0;31m'
CYAN='\033[1;36m'
GREEN='\033[1;32m'
NC='\033[0m'
S3_PATH=$1

# Run the Streamlit app and save the output to "temp.txt"
streamlit run summarize_app.py > temp.txt & 

# Read the text file using cat
echo "Getting the URL to view your Streamlit app in the browser"

# Extract the last four digits of the port number from the Network URL
sleep 5
PORT=$(grep "Network URL" temp.txt | awk -F':' '{print $NF}' | awk '{print $1}' | tail -c 5)
echo -e "${CYAN}${CURRENTDATE}: [INFO]:${NC} Port Number ${PORT}" 


# Get Studio domain information
DOMAIN_ID=$(jq .DomainId /opt/ml/metadata/resource-metadata.json || exit 1)
RESOURCE_NAME=$(jq .ResourceName /opt/ml/metadata/resource-metadata.json || exit 1)
RESOURCE_ARN=$(jq .ResourceArn /opt/ml/metadata/resource-metadata.json || exit 1)

# Remove quotes from string
DOMAIN_ID=`sed -e 's/^"//' -e 's/"$//' <<< "$DOMAIN_ID"`
RESOURCE_NAME=`sed -e 's/^"//' -e 's/"$//' <<< "$RESOURCE_NAME"`
RESOURCE_ARN=`sed -e 's/^"//' -e 's/"$//' <<< "$RESOURCE_ARN"`
RESOURCE_ARN_ARRAY=($(echo "$RESOURCE_ARN" | tr ':' '\n'))

# Get Studio domain region
REGION=$(echo "${RESOURCE_ARN_ARRAY[3]}")

# Check if it's Collaborative Space
SPACE_NAME=$(jq .SpaceName /opt/ml/metadata/resource-metadata.json || exit 1)

# if it's not a collaborative space 
if [ -z "$SPACE_NAME" ] || [ $SPACE_NAME == "null" ] ;
then
    # If it's a user-profile access
    echo -e "${CYAN}${CURRENTDATE}: [INFO]:${NC} Domain Id ${DOMAIN_ID}"
    STUDIO_URL="https://${DOMAIN_ID}.studio.${REGION}.sagemaker.aws"
    
# It is a collaborative space
else

    SEM=true
    SPACE_ID=

    # Check if Space Id was previously configured
    if [ -f /tmp/space-metadata.json ]; then
        SAVED_SPACE_ID=$(jq .SpaceId /tmp/space-metadata.json || exit 1)
        SAVED_SPACE_ID=`sed -e 's/^"//' -e 's/"$//' <<< "$SAVED_SPACE_ID"`

        if [ -z "$SAVED_SPACE_ID" ] || [ $SAVED_SPACE_ID == "null" ]; then
            ASK_INPUT=true
        else
            ASK_INPUT=false
        fi
    else
        ASK_INPUT=true
    fi

    # If Space Id is not available, ask for it
    while [[ $SPACE_ID = "" ]] ; do
        # If Space Id already configured, skeep the ask
        if [ "$ASK_INPUT" = true ]; then
            echo -e "${CYAN}${CURRENTDATE}: [INFO]:${NC} Please insert the Space Id from your url. e.g. https://${GREEN}<SPACE_ID>${NC}.studio.${REGION}.sagemaker.aws/jupyter/default/lab"
            read SPACE_ID
            SEM=true
        else
            SPACE_ID=$SAVED_SPACE_ID
        fi

        if ! [ -z "$SPACE_ID" ] && ! [ $SPACE_ID == "null" ] ;
        then
            while $SEM; do
                echo "${SPACE_ID}"
                read -p "Should this be used as Space Id? (y/N) " yn
                case $yn in
                    [Yy]* )
                        echo -e "${CYAN}${CURRENTDATE}: [INFO]:${NC} Domain Id ${DOMAIN_ID}"
                        echo -e "${CYAN}${CURRENTDATE}: [INFO]:${NC} Space Id ${SPACE_ID}"

                        jq -n --arg space_id $SPACE_ID '{"SpaceId":$space_id}' > /tmp/space-metadata.json

                        STUDIO_URL="https://${SPACE_ID}.studio.${REGION}.sagemaker.aws"

                        SEM=false
                        ;;
                    [Nn]* ) 
                        SPACE_ID=
                        ASK_INPUT=true
                        SEM=false
                        ;;
                    * ) echo "Please answer yes or no.";;
                esac
            done
        fi
    done
fi

echo -e "${CYAN}${CURRENTDATE}: [INFO]:${NC} Studio Url ${STUDIO_URL}"


link="${STUDIO_URL}/jupyter/${RESOURCE_NAME}/proxy/${PORT}/"

echo -e "${CYAN}${CURRENTDATE}: [INFO]:${NC} Starting Streamlit App"
echo -e "${CYAN}${CURRENTDATE}: [INFO]: ${GREEN}${link}${NC}"

exit 0
fi



Overwriting summ-run.sh


## (2) Streamlit 어플리케이션 실행하기

1. listing 어플리케이션 수행을 위한 디렉토리 경로로 이동합니다.
    
    cd /aws-gen-for-retail/l_lab
    
2. Streamlit 수행을 위한 라이브러리를 설치합니다. 

    . summ-setup.sh

3. Streamlit 실행을 위한 스크립트를 수행합니다. 

    . summ-run.sh 

4. 스크립트 수행 결과 생성되는 링크를 클릭하면 Streamlit 어플리케이션을 시작할 수 있습니다. 

5. '2022-Shareholder-Letter-ko.pdf'를 1_lab 폴더에서 다운로드한 후, 실행 중인 어플리케이션에서 해당 파일을 업로드 합니다.

![bedrock_langchian_image](./image/summ-web1.png)


6. '중간단계' 요약하기를 클릭한 후, '요약하기' 버튼을 클릭합니다. 

![bedrock_langchian_image](./image/summ-web2.png)



# 추가 실습

## STUFF 
'Stuff(스터핑)' 체인은 스터핑이라는 더 간단한 접근 방식을 활용합니다. 이 접근 방식에서는 프롬프트가 모든 관련 데이터를 컨텍스트로서 언어 모델에 전달합니다. 이 접근 방식은 작은 데이터 조각에는 효과적이지만 많은 데이터를 처리할 때는 비실용적입니다.

In [11]:
from langchain.chains.summarize import load_summarize_chain
import textwrap


stuff_prompt_template = "{text}\n\n위의 내용을 Korean으로 bullet point 3개로 요약합니다:"
stuff_prompt = PromptTemplate(template=stuff_prompt_template, input_variables=["text"])

chain = load_summarize_chain(llm, chain_type="stuff", prompt=stuff_prompt,verbose = True)

output_summary = chain.run(docs) 

wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
CEO로서 두 번째 연례 주주 서한을 작성하기 위해 자리에 앉았을 때 저는 Amazon의 앞날에 대해 낙관적이고 활력을 얻었습니다. 2022년은 최근 기억에 있어 어려운 거시경제적 해 중 하나이고 우리 자체의 운영상의 어려움에도 불구하고 우리는 여전히 수요를 늘릴 방법을 찾았습니다(팬데믹 전반기에 경험한 전례 없는 성장에 더해). 우리는 장단기적으로 고객 경험을 의미 있게 개선하기 위해 대규모 사업을 혁신했습니다. 그리고 고객, 주주 및 직원을 위해 Amazon의 미래를 바꿀 수 있다고 믿는 장기 투자를 계속 유지하면서 투자 결정과 앞으로 나아갈 방법에 중요한 조정을 했습니다.  작년에 이례적인 수의 동시 도전 과제가 있었지만 현실은 유능하고 자금이 충분한 경쟁자가 많은 크고 역동적인 글로벌 시장 부문에서 운영하는 경우(아마존이 모든 비즈니스를 운영하는 조건) 조건이 오랫동안 정체되어 있는 경우는 드뭅니다.  제가 Amazon에 근무한 25년 동안 끊임없는 변화가 있었고 그 중 대부분은 우리가 스스로 시작했습니다. 내가 1997년에 Amazon에 입사했을 때 우리는 1996년에 1,500만 달러의 매출을 올렸고, 도서 전용 소매업체였으며, 제3자 시장이 없었고, 미국 내 주소로만 배송되었습니다. 오늘날 Amazon은 단위 판매의 60%를 차지하고 전 세계 거의 모든 국가의 고객에게 도달하는 활기찬 타사 판매자 에코시스템을 통해 상상할 수 있는 거의 모든 물리적 및 디지털 소매 품목을 판매합니다. 마찬가지로 클라우드에서 일련의 기술 인프라 서비스를 중심으로 비즈니스를 구축하는 것은 AWS를 추구하기 시작한 2003년에는 분명하지 않았으며 2006년 첫 서비스를 출시했을 때도 마찬가지였습니다. 거의 모든 책을 60초 안에 손끝에서 볼 수 있습니다 2007년 Kindle을 출시했

## REFINE

'Refine(정제)' 체인에는 첫 번째 데이터 청크에 대한 초기 프롬프트가 포함되어 출력을 생성합니다. 언어 모델은 이 출력을 다음 문서와 함께 전달하여 새 문서를 기반으로 출력을 다듬습니다.

이 반복적인 정제 프로세스는 보다 정확한 요약을 생성하는 데 도움이 됩니다. load_summarize_chain 함수를 사용하여 'refine' 체인을 설정하고 요약된 출력을 얻습니다.

In [12]:
from langchain.chains.summarize import load_summarize_chain

prompt_template = "{text}\n\n위의 내용을 Korean으로 요약합니다:"
prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

refine_template = "{text}\n\n위의 내용을 Korean으로 최종적으로 요약합니다:"
refine_prompt = PromptTemplate(template=refine_template, input_variables=["text"])

chain = load_summarize_chain(
    llm, chain_type="refine", 
    question_prompt=prompt, 
    refine_prompt=refine_prompt,
    verbose=True)

chain.run(docs) 



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
CEO로서 두 번째 연례 주주 서한을 작성하기 위해 자리에 앉았을 때 저는 Amazon의 앞날에 대해 낙관적이고 활력을 얻었습니다. 2022년은 최근 기억에 있어 어려운 거시경제적 해 중 하나이고 우리 자체의 운영상의 어려움에도 불구하고 우리는 여전히 수요를 늘릴 방법을 찾았습니다(팬데믹 전반기에 경험한 전례 없는 성장에 더해). 우리는 장단기적으로 고객 경험을 의미 있게 개선하기 위해 대규모 사업을 혁신했습니다. 그리고 고객, 주주 및 직원을 위해 Amazon의 미래를 바꿀 수 있다고 믿는 장기 투자를 계속 유지하면서 투자 결정과 앞으로 나아갈 방법에 중요한 조정을 했습니다.  작년에 이례적인 수의 동시 도전 과제가 있었지만 현실은 유능하고 자금이 충분한 경쟁자가 많은 크고 역동적인 글로벌 시장 부문에서 운영하는 경우(아마존이 모든 비즈니스를 운영하는 조건) 조건이 오랫동안 정체되어 있는 경우는 드뭅니다.  제가 Amazon에 근무한 25년 동안 끊임없는 변화가 있었고 그 중 대부분은 우리가 스스로 시작했습니다. 내가 1997년에 Amazon에 입사했을 때 우리는 1996년에 1,500만 달러의 매출을 올렸고, 도서 전용 소매업체였으며, 제3자 시장이 없었고, 미국 내 주소로만 배송되었습니다. 오늘날 Amazon은 단위 판매의 60%를 차지하고 전 세계 거의 모든 국가의 고객에게 도달하는 활기찬 타사 판매자 에코시스템을 통해 상상할 수 있는 거의 모든 물리적 및 디지털 소매 품목을 판매합니다. 마찬가지로 클라우드에서 일련의 기술 인프라 서비스를 중심으로 비즈니스를 구축하는 것은 AWS를 추구하기 시작한 2003년에는 분명하지 않았으며 2006년 첫 서비스를 출시했을 때도 마찬가지였습니다. 거의 모든 책을 60초 안에 손끝에서 볼 수 있습니다 2007년 Kindle을 출시

' 아마존의 미래 전망에 대해 다음과 같이 요약할 수 있습니다:\n\n- 기계학습과 제너레이티브 AI 기술이 고객 경험을 크게 변화시킬 것으로 예상됨\n\n- AWS를 통해 기계학습 칩과 인프라를 저렴하게 제공, 모든 규모 기업이 AI 활용 가능\n\n- 아직 소매와 IT 시장의 대부분이 온프레미스에 있어 성장 여지가 많음\n\n- 고객 중심의 혁신과 노력으로 향후 몇 년 간 상당한 성장이 기대됨\n\n- 아마존은 기술 선도와 꾸준한 발명을 통해 어려운 경제 환경에서도 성공 가능할 것\n\n요약하면 아마존은 AI/클라우드 기술 리더십을 바탕으로 시장 점유율 확대와 지속 성장을 이룰 것으로 낙관하고 있다는 것입니다.'